<a href="https://colab.research.google.com/github/AlibekAdilzhan/ml_models/blob/main/my_gradient_boosting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
from sklearn.tree import DecisionTreeRegressor

In [ ]:
df = pd.read_csv('sample_data/california_housing_train.csv')
df = df.dropna()
df.describe()
df_test = df[15000 : 17000]
df = df[ : 15000]
# np.where(np.isnan(df))
df.describe()

In [ ]:
X_train = df.drop(["median_house_value"], axis = 1)
Y_train = df["median_house_value"]
# type(X_train["Age"])
X_train

In [ ]:
Y_train = Y_train.apply(lambda x: 1 if x > 350000 else -1)

In [ ]:
from copy import deepcopy

In [ ]:
def loss_fn(y, f):
    lf = np.log(1 + np.exp(-2 * y * f))

    return lf

def dLdf(y, f):
    dlf = 1 / (1 + np.exp(-2 * y * f)) * (-2 * y) * np.exp(-2 * y * f)
    return dlf

def find_gamma(Y):
    gamma = np.random.rand()
    alpha = 0.1 # learning rate for finding gamma
    epsilon = 0.001

    while True:
        gamma_previous = gamma
        dL_theta = 0

        dL = dLdf(Y, gamma)
        dL_theta = np.sum(dL) / len(Y)
        # print(dL_theta, 'this is the dL_theta', gamma)
        gamma = gamma - alpha * dL_theta
        # print(gamma_previous, gamma)
        if abs(gamma_previous - gamma) < epsilon:
            break

    return gamma


def model_output(models, rhos, x):
    output = rhos[0]

    for i in range(1, len(models)):
        model = models[i]
        prediction = model.predict(x)
        output += prediction * rhos[i]

    return output

def find_rho(models, rhos, new_model, X, Y):
    rho = np.random.rand()
    alpha = 0.001 # learning rate for finding rho
    epsilon = 0.0001

    while True:
        rho_previous = rho
        # dL_theta = 0
        y_bar = model_output(models, rhos, X)
        prediction = new_model.predict(X)
        y_bar_new = y_bar + prediction * rho
        # print(y_bar_new.shape)

        z = dLdf(Y, y_bar_new) * prediction
        dL_theta = np.sum(z) / len(Y)

        rho = rho - alpha * dL_theta
        print(rho)
        if abs(rho - rho_previous) < epsilon:
            break
    
    return rho


def GBM(X, Y, num_iterations):
    models = []
    rhos = []
    gamma = find_gamma(Y)
    models.append(1)
    rhos.append(gamma)
    # print(gamma)
    # print('haha')

    for i in range(num_iterations):
        # print(i)
        if i == 0:
            y_bar = gamma
        else:
            y_bar = model_output(models, rhos, X)

        # for j in range(len(residuals)):
        residuals = -dLdf(Y, y_bar)
        regression_tree = DecisionTreeRegressor(max_depth = 3)
        regression_tree.fit(X, residuals)

        # print('gege', i)
        rho = find_rho(models, rhos, regression_tree, X, Y)
        # print(rho)

        models.append(deepcopy(regression_tree))
        rhos.append(rho)
    
    return models, rhos
        
        

In [ ]:
models, rhos = GBM(X_train, Y_train, 50)

In [ ]:
np.random.rand()

In [ ]:
# df_test = pd.read_csv('sample_data/california_housing_test.csv')
X_test = df_test.drop('median_house_value', axis = 1)
Y_test = df_test['median_house_value']
Y_test = Y_test.apply(lambda x: 1 if x > 350000 else -1)
Y_test

In [ ]:
prediction = np.zeros(Y_test.shape).astype(float)

prediction = model_output(models, rhos, X_test)

In [ ]:
prediction_category = np.zeros(prediction.shape)

for i in range(len(Y_test)):
    if prediction[i] >= 0:
        prediction_category[i] = 1
    else:
        prediction_category[i] = -1

In [ ]:
results = (prediction_category == Y_test).astype(int)
print(np.sum(results) / len(Y_test))

In [ ]:
prediction.shape

In [ ]:
for i in prediction:
    print(i)

In [ ]:
np.sum(Y_test < 0) / len(Y_test)

In [ ]:
np.sum(prediction > 0)